In [1]:
# Import modules
import datetime
import glob

# Import installed / 3rd party modules
import imageio
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pathlib
import spiceypy
import tqdm

# Load the SPICE kernels via a meta file
spiceypy.furnsh('kernel_meta.txt')

In [2]:
# We want to compute the Solar System barycentre (SSB) w.r.t to the centre of
# the Sun for a certain time interval
# First, we set an initial time in UTC.
init_time_utc = datetime.datetime(year=2000, month=1, day=1, \
                                  hour=0, minute=0, second=0)

# Add a number of days; you can play around with the datetime variables; but
# leave it as it is for the first try, since other computations and comments
# are based on this value.
delta_days = 10000
end_time_utc = init_time_utc + datetime.timedelta(days=delta_days)

# Convert the datetime objects now to strings
init_time_utc_str = init_time_utc.strftime('%Y-%m-%dT%H:%M:%S')
end_time_utc_str = end_time_utc.strftime('%Y-%m-%dT%H:%M:%S')

# Print the starting and end times
print('Init time in UTC: %s' % init_time_utc_str)
print('End time in UTC: %s\n' % end_time_utc_str)

# Convert to Ephemeris Time (ET) using the SPICE function utc2et.
init_time_et = spiceypy.utc2et(init_time_utc_str)
end_time_et = spiceypy.utc2et(end_time_utc_str)

Init time in UTC: 2000-01-01T00:00:00
End time in UTC: 2027-05-19T00:00:00



In [3]:
# Now we compute the position of the Solar System's barycentre w.r.t. our Sun:
# First we set an empty list that stores later all x, y, z components for each
# time step
ssb_wrt_sun_position = []

# What shall the time step be? Given in seconds
time_delta = 864000 * 2 # 864000 corresponds to 10 days

# Time interval
time_interval_et = np.arange(init_time_et, end_time_et, time_delta)

In [4]:
# We will use the Sun Radius to scale the values
_, radii_sun = spiceypy.bodvcd(bodyid=10, item='RADII', maxn=3)

radius_sun = radii_sun[0]

In [5]:
# Create a temporary folder to store all figures
pathlib.Path('temp/').mkdir(parents=True, exist_ok=True)

# Set font properties
font = {'size': 10}

matplotlib.rc('font', **font)

# Set a dark background... since... space is dark
plt.style.use('dark_background')

# Set placeholders for the ecliptic longitude and distance values
ssb_ecl_long = []
ssb_dist = []

# Iterate through all ET times
for index, et_it in tqdm.tqdm(enumerate(time_interval_et)):

    # Create the plot with its content
    plt.clf()

    # Create a figure + axes in polar coordiantes
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(7, 7))
    
    # Create a yellow circle that represents the Sun, add it to the ax
    sun_circ = plt.Circle((0.0, 0.0), 1, transform=ax.transData._b, color="yellow", alpha=0.8)
    ax.add_artist(sun_circ)
    
    # Compute the SSB position (0) as seen from the Sun (10)
    _ssb_position, _ = spiceypy.spkgps(targ=0,
                                       et=et_it, \
                                       ref='ECLIPJ2000',
                                       obs=10)

    # Convert the vector to polar coordinates
    _dist, _ecl_long, _ = spiceypy.recrad(_ssb_position)

    
    # Append the results to the final list. Scale the distance using the Sun's radius
    ssb_ecl_long.append(_ecl_long)
    ssb_dist.append(_dist / radius_sun)

    # Plot the current SSB position. If the list contains more than 1 value, plot a dashed line,
    # representing previous results
    ax.plot(ssb_ecl_long[-1], ssb_dist[-1], marker='o', color="forestgreen", markersize=4)
    if len(ssb_ecl_long) > 2:
        ax.plot(ssb_ecl_long, ssb_dist, linestyle='--', color="forestgreen", linewidth=2)

    # Let's display the Vernal Equinox (only)
    _, _ = plt.thetagrids(range(0, 360, 90), ('Vernal\n Equinox', '', '', ''),
                          multialignment="center")
        
    # Create now vectors that indicate the direction of some chosen planets:
    for planet_dict in [{"planet": "Jupiter", "NAIF-ID": 5, "color": "tab:purple"},
                        {"planet": "Saturn", "NAIF-ID": 6, "color": "tab:olive"}]:
        
        # Compute the planet's position
        planet_position, _ = spiceypy.spkgps(targ=planet_dict["NAIF-ID"], et=et_it,
                                             ref="ECLIPJ2000", obs=10)
    
        # Convert to the ecliptic longitude for the polar plot
        _, _planet_ecl_long, _ = spiceypy.recrad(planet_position)
    
        # Use only the longitude to indicate the pointing direction of the planet
        plt.arrow(_planet_ecl_long, 1.5, 0, 1,
                  alpha = 0.8, width = 0.015,
                  edgecolor = planet_dict["color"], facecolor = planet_dict["color"], lw = 1,
                  label=planet_dict["planet"])

    # Set a legend outside the plot
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), fancybox=True, shadow=True, ncol=3)
    
    # Set a title that contains the datetime
    plt.title("Position of the Solar System Barycenter (green) at: " \
              + f"{spiceypy.et2datetime(et_it).strftime('%Y %j')}")

    # Set some plotting parameters
    ax.set_aspect('equal')
    ax.set_rmax(3)
    ax.grid(False)
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.spines["polar"].set_visible(False)

    # Saving the figure in high quality.
    plt.savefig(f'temp/SSB_WRT_SUN_{str(index).zfill(4)}.png', dpi=300)
    
    # Close
    plt.close()

501it [02:44,  3.05it/s]


<Figure size 432x288 with 0 Axes>

In [6]:
# In this part all figures are merged into one animation (GIF)

# Get all filepaths
file_names = sorted(glob.glob('temp/*.png'))

# Set an empty list that will contain all images
ssb_images = []

# Iterate through the list of image paths, read the image with the imageio
# library and append the result to the list
for figure_f in tqdm.tqdm(file_names):
    ssb_images.append(imageio.imread(figure_f))

100%|█████████████████████████████████████████| 501/501 [00:42<00:00, 11.68it/s]


In [7]:
# Save the list of images as a GIF. The duration of a single image is given
# in seconds
imageio.mimsave('ssb_movement.gif', ssb_images, duration=0.015)